# Het verschil tussen startpositie en strategie

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
from collections import defaultdict
pyo.init_notebook_mode(connected=True)

plt.style.use('default')
sns.set_palette("viridis")


df = pd.read_csv('merged_f1_data_1994_2022.csv')

df['Pos_numeric'] = pd.to_numeric(df['Pos'], errors='coerce')
df['FinPos_numeric'] = pd.to_numeric(df['FinPos'], errors='coerce')

valid_data = df[
    (df['Pos_numeric'].notna()) & 
    (df['FinPos_numeric'].notna()) & 
    (df['Pos_numeric'] > 0) & 
    (df['FinPos_numeric'] > 0)
].copy()

max_start_pos = int(valid_data['Pos_numeric'].max())
max_finish_pos = int(valid_data['FinPos_numeric'].max())

heatmap_data = np.zeros((max_finish_pos, max_start_pos))

for _, row in valid_data.iterrows():
    start_pos = int(row['Pos_numeric']) - 1  # -1 voor 0-indexing
    finish_pos = int(row['FinPos_numeric']) - 1  # -1 voor 0-indexing
    heatmap_data[finish_pos, start_pos] += 1

heatmap_df = pd.DataFrame(
    heatmap_data,
    index=range(1, max_finish_pos + 1),  # Eindposities
    columns=range(1, max_start_pos + 1)  # Startposities
)

fig = go.Figure(data=go.Heatmap(
    z=heatmap_df.values,
    x=heatmap_df.columns,  # Startposities
    y=heatmap_df.index,    # Eindposities
    colorscale='Viridis',
    showscale=True,
    colorbar=dict(
        title="Aantal races"
    ),
    hovertemplate=
    '<b>Start Positie:</b> P%{x}<br>' +
    '<b>Eind Positie:</b> P%{y}<br>' +
    '<b>Aantal races:</b> %{z}<br>' +
    '<extra></extra>'  # Verwijdert de standaard trace box
))

fig.update_layout(
    title={
        'text': 'F1 Startpositie vs Eindpositie (1994-2022)<br>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    xaxis=dict(
        title='Startpositie (Pos)',
        title_font=dict(size=14),
        tickmode='linear',
        tick0=1,
        dtick=1
    ),
    yaxis=dict(
        title='Eindpositie (FinPos)',
        title_font=dict(size=14),
        tickmode='linear',
        tick0=1,
        dtick=1
    ),
    width=1000,
    height=700,
    font=dict(size=12)
)

fig.show()

pole_wins = heatmap_data[0, 0]  # [eindpositie-1, startpositie-1]
total_pole_starts = heatmap_data[:, 0].sum()
pole_win_percentage = (pole_wins / total_pole_starts) * 100

print(f"\nTop 5 meest voorkomende combinaties:")
top_combinations = []
for i in range(heatmap_data.shape[0]):
    for j in range(heatmap_data.shape[1]):
        if heatmap_data[i, j] > 0:
            top_combinations.append((i+1, j+1, int(heatmap_data[i, j])))

top_combinations.sort(key=lambda x: x[2], reverse=True)

for i, (finish_pos, start_pos, count) in enumerate(top_combinations[:5]):
    print(f"{i+1}. Start P{start_pos} → Finish P{finish_pos}: {count} keer")

print(f"\nWin rates per startpositie (top 10):")
for start_pos in range(1, min(11, max_start_pos + 1)):
    wins = heatmap_data[0, start_pos-1]  # Eindpositie 1 (index 0)
    total_starts = heatmap_data[:, start_pos-1].sum()
    if total_starts > 0:
        win_rate = (wins / total_starts) * 100
        print(f"P{start_pos}: {win_rate:.1f}% ({int(wins)}/{int(total_starts)})")

valid_data['position_change'] = valid_data['Pos_numeric'] - valid_data['FinPos_numeric']

fig_hist = go.Figure()

# Histogram
hist_data, bin_edges = np.histogram(valid_data['position_change'], bins=range(-20, 22))

x_values = list(range(-20, 21))

fig_hist.add_trace(go.Bar(
    x=x_values,
    y=hist_data,
    name='Frequentie',
    marker_color='skyblue',
    marker_line_color='black',
    marker_line_width=1,
    hovertemplate=
    '<b>Positie verandering:</b> %{x}<br>' +
    '<b>Aantal races:</b> %{y}<br>' +
    '<extra></extra>'
))

fig_hist.add_vline(
    x=0, 
    line_dash="dash", 
    line_color="red",
    line_width=2,
    annotation_text="",
    annotation_position="top"
)

fig_hist.update_layout(
    title={
        'text': 'Verdeling van Positie Veranderingen in F1 Races (1994-2022)<br>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    xaxis=dict(
        title='Positie Verandering',
        title_font=dict(size=14),
        tickmode='linear',
        dtick=2
    ),
    yaxis=dict(
        title='Frequentie',
        title_font=dict(size=14)
    ),
    width=900,
    height=500,
    showlegend=False,
    font=dict(size=12)
)

fig_hist.show()



Top 5 meest voorkomende combinaties:
1. Start P1 → Finish P1: 252 keer
2. Start P2 → Finish P1: 140 keer
3. Start P2 → Finish P2: 122 keer
4. Start P3 → Finish P3: 113 keer
5. Start P3 → Finish P2: 101 keer

Win rates per startpositie (top 10):
P1: 55.9% (252/451)
P2: 31.0% (140/451)
P3: 14.2% (64/451)
P4: 5.2% (23/440)
P5: 3.4% (14/409)
P6: 2.7% (11/414)
P7: 1.7% (7/413)
P8: 1.0% (4/402)
P9: 0.0% (0/405)
P10: 1.5% (6/403)
